# 3rd Level Model Structure: Single Stage Reactive Distillation

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets

In [2]:
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,1)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Linking Stage Variables

### Vapor Between Reactive Stages

In [6]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [7]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [8]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [9]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

# Fixing Redundent Stream Variables

In [10]:
# condenser
model.condenser.VLE_block.n_ave.fix(4)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [11]:
# 'reboiler' fixing last stage V_in

model.reactive[model.TRAY.last()].V['in'].fix(0)
for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

# Load Operating Parameters

In [12]:
# condenser
model.condenser.P.fix(19)
# model.condenser.T.fix(30+273.15)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(5000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2)-0.002/2)
    model.reactive[j].z['H2'].fix(2/(1+2)-0.002/2)
    model.reactive[j].z['C30H62'].fix(0.002)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    # model.reactive[j].T.setlb(300+273.15)

In [13]:
check_DOF(pe,model)

Active Equality Constraints:	 2300
Active Inequality Constraints:	 0
Active Variables:		 2483
Fixed Variables:		 182
DOF:				 1


In [14]:
model.obj = pe.Objective(expr = model.reactive[1].T ,sense=pe.maximize)

In [15]:
opt = pe.SolverFactory('ipopt')
opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'
# opt.options['nlp_scaling_method'] = None
# opt.options['constr_viol_tol'] = 1e-7
# opt.options['acceptable_constr_viol_tol'] = 1e-7
opt.options['max_iter'] = 7000
# opt.options['dual_inf_tol'] = '+inf'
# opt.options['acceptable_dual_inf_tol'] = '+inf'

In [16]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [17]:
results = opt.solve(model,tee=True)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
max_iter=7000


List of user-set options:

                                    Name   Value                used
                           linear_solver = ma86                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma86.

Number of nonzeros in equality constraint Jacobian...:     7301
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.....

 145 -5.1806951e+02 1.43e+01 1.95e+03  -1.0 6.59e+05    -  4.30e-05 3.47e-07h  8
 146 -5.1806962e+02 1.43e+01 1.95e+03  -1.0 6.45e+05    -  5.07e-05 7.30e-07h  7
 147 -5.1806984e+02 1.43e+01 1.95e+03  -1.0 6.25e+05    -  5.94e-05 1.55e-06f  6
 148 -5.1806990e+02 1.43e+01 1.95e+03  -1.0 5.98e+05    -  8.90e-05 4.02e-07h  8
 149r-5.1806990e+02 1.43e+01 9.99e+02   1.2 0.00e+00    -  0.00e+00 3.88e-10R 18
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 150r-5.1806985e+02 1.43e+01 4.98e+03   1.2 1.79e+03    -  2.99e-03 3.18e-05f  1
 151r-5.1805549e+02 1.42e+01 4.62e+03   0.5 6.21e+02    -  5.59e-02 6.42e-03f  1
 152r-5.1805974e+02 1.41e+01 4.34e+03  -0.2 1.12e+02    -  2.07e-01 6.23e-02f  1
 153r-5.1809485e+02 1.28e+01 3.00e+03  -0.2 2.43e+02    -  4.18e-01 3.13e-01f  1
 154 -5.1810318e+02 1.28e+01 2.83e+01  -1.0 1.26e+05    -  2.22e-04 6.51e-05h  5
 155 -5.1810743e+02 1.28e+01 3.11e+01  -1.0 1.00e+05    -  3.55e-04 3.61e-05h  6
 156 -5.1810934e+02 1.28e+01

 238 -5.4464688e+02 5.78e+00 1.65e+07  -5.7 1.77e+03    -  9.40e-01 3.84e-06h  1
 239 -5.4466290e+02 5.77e+00 1.65e+07  -5.7 1.77e+03    -  2.27e-05 1.63e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 240 -5.4541785e+02 5.58e+00 1.65e+07  -5.7 1.77e+03    -  6.39e-05 7.69e-02f  1
 241 -5.4541785e+02 5.58e+00 1.82e+07  -5.7 3.77e+01    -  9.99e-06 2.48e-06h  2
 242r-5.4541785e+02 5.58e+00 1.00e+03   0.7 0.00e+00    -  0.00e+00 3.90e-08R  7
 243r-5.4540879e+02 5.57e+00 9.99e+02   0.7 1.09e+06    -  8.53e-05 1.48e-06f  1
 244r-5.4540597e+02 4.90e+00 1.49e+03   0.7 9.71e+03    -  9.71e-06 1.32e-04f  1
 245 -5.4540732e+02 4.83e+00 1.02e+00  -5.7 2.14e+01    -  8.49e-03 1.48e-02h  1
 246 -5.4540713e+02 4.75e+00 5.75e+00  -5.7 7.02e+00    -  2.40e-02 1.58e-02h  1
 247r-5.4540713e+02 4.75e+00 9.99e+02   0.6 0.00e+00    -  0.00e+00 4.26e-08R  2
 248r-5.4540713e+02 4.75e+00 9.99e+02   0.6 4.51e+03    -  4.44e-06 8.98e-08f  1
 249r-5.4540704e+02 4.72e+00

 335 -5.6119008e+02 6.41e-03 1.68e+10  -5.7 4.30e-01  10.5 8.39e-09 1.99e-03f  3
 336r-5.6119008e+02 6.41e-03 1.00e+03  -2.3 0.00e+00  11.0 0.00e+00 2.64e-07R 15
 337r-5.6118905e+02 7.20e-04 9.98e+02  -2.3 1.97e+02    -  2.30e-03 9.90e-04f  1
 338 -5.6118933e+02 7.18e-04 7.79e+02  -5.7 2.40e+01    -  9.91e-01 2.30e-03h  1
 339 -5.6118941e+02 6.28e-04 3.54e+04  -5.7 1.13e-01    -  7.79e-01 1.25e-01f  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 340 -5.6118997e+02 1.99e-04 1.93e+04  -5.7 1.61e-01    -  1.00e+00 6.83e-01h  1
 341 -5.6119261e+02 3.30e-06 4.66e+03  -5.7 5.14e-01    -  1.00e+00 1.00e+00f  1
 342 -5.6119261e+02 2.37e-06 3.32e+03  -5.7 3.52e-08  10.5 1.00e+00 2.84e-01h  1
 343 -5.6119317e+02 2.50e-06 5.94e+04  -5.7 1.64e+01    -  1.00e+00 6.65e-03f  1
 344 -5.6125117e+02 1.60e-03 1.68e+04  -5.7 1.13e+01    -  1.11e-01 1.00e+00f  1
 345 -5.6123744e+02 1.59e-03 9.75e+05  -5.7 4.28e+01    -  4.96e-03 6.25e-02h  5
 346 -5.6121789e+02 1.57e-03

In [18]:
update_dual(pe,model)

In [19]:
beautify2(pe,model)

Here comes the result:
----------------------------------------------------------------------------------------------------
		T		 Q			 V			 L
               30.00 	       -17.6521277784 	         0.1898639734 	         0.0000000000
              300.00 	       -46.6371828120 	         0.4598211652 	         0.0011329672
----------------------------------------------------------------------------------------------------
Top
V	 0.18986397340734695
L	 0.012365437982382972
W	 0.2575917538485565
----------------------------------------------------------------------------------------------------
Bottom
L	 0.0011329672358523358
----------------------------------------------------------------------------------------------------
Condenser:	Vapor		Liquid		Last Stage	Vapor		Liquid
H2 		3.412%		0.045%		 H2       	1.410%		0.068%
CO 		0.058%		0.002%		 CO       	0.024%		0.001%
CO2 		19.268%		3.744%		 CO2      	8.057%		0.802%
H2O 		0.223%		0.000%		 H2O      	56.112%		8.549%
C2H4 		13.133%		4.102%		 C2

In [20]:
model.reactive[1].kinetics_block.r_FT_total.value

0.2951681295456062

In [21]:
# model.solutions.store_to(results)
# with open('../saved_solutions/1_stage_condenser_adiabatic.pickle','wb') as f:
#     pickle.dump(results,f)

### Check non-active constraints (dual = 0)

In [22]:
for i in model.dual:
    if abs(model.dual[i]) == 0:
        print('{:40s}'.format(str(i)),'\t\t',model.dual[i])

reactive[1].heat_balance_main_con        		 0.0
Lh_condenser_con                         		 0.0
reactive[1].energy_block.dH_F_con[C30H62] 		 0.0
reactive[1].energy_block.dH_F_con[CO]    		 0.0
reactive[1].energy_block.dH_F_con[H2]    		 0.0
reactive[1].energy_block.H_F_con         		 0.0
condenser.energy_block.dH_F_con[C30H62]  		 0.0
condenser.energy_block.dH_F_con[CO]      		 0.0
condenser.energy_block.dH_F_con[H2]      		 0.0
condenser.energy_block.H_F_con           		 0.0


# Iterative Data Analysis

In [23]:
opt.options['linear_solver'] = 'ma86'
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [24]:
results_anchor = results
update_dual(pe,model)
model.solutions.store_to(results_anchor)

In [25]:
Refluxrange = np.linspace(0,2,21)
Trange = np.linspace(300+273.15,200+273.15,11)

In [26]:
rf_data_master = {}

for t in Trange:
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = model.reactive[1].T,sense=pe.maximize)
    model.solutions.load_from(results_anchor)
    update_dual(pe,model)
    model.reactive[1].T.fixed = False
    model.reactive[1].T.setub(t)
    model.condenser.L['out'].fix(0)
    results_anchor = opt.solve(model,tee=False)
    model.solutions.store_to(results_anchor)
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = model.condenser.L['out'],sense=pe.maximize)
    print('\nWorking on T = {} K'.format(model.reactive[1].T.value))
    print('-'*108)

    rf_data = {}
    rf_data['r'] = {}; rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; rf_data['Re'] = []; rf_data['D'] = []
    rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = []; rf_data['r_WGS'] = []; rf_data['r_FT'] = []
    rf_data['g'] = {}; rf_data['d'] = {}; rf_data['b'] = {};
    
    for i in model.reactive[1].r_total_comp:
        rf_data['r'][i] = []
        rf_data['g'][i] = []
        rf_data['d'][i] = []
        rf_data['b'][i] = []       

    for re in Refluxrange:
        model.condenser.L['out'].fixed = False
        model.condenser.L['out'].setub(re)
        model.reactive[1].T.fix(t)
        results = opt.solve(model,tee=False)
        update_dual(pe,model)
        print('Solved\t|Reflux = {:.3f} kmol/s\t|Vapor = {:.3f} kmol/s\t|Distillate = {:.3f} kmol/s\t|Bottom = {:.3f} kmol/s'.\
              format(model.condenser.L['out'].value,model.condenser.V['out'].value,model.condenser.L['P'].value,model.reactive[1].L['out'].value))
        rf_data['T'].append(model.reactive[1].T.value)
        rf_data['Q'].append(model.reactive[1].Q_main.value)
        rf_data['V'].append(model.condenser.V['out'].value)
        rf_data['Re'].append(model.condenser.L['out'].value)
        rf_data['D'].append(model.reactive[1].L['P'].value)

        for i in model.reactive[1].r_total_comp:
            rf_data['r'][i].append(model.reactive[1].r_total_comp[i].value)
            rf_data['g'][i].append(model.condenser.y[i].value*model.condenser.V['out'].value)
            rf_data['d'][i].append(model.condenser.x[i].value*model.condenser.L['P'].value)
            rf_data['b'][i].append(model.reactive[1].x[i].value*model.reactive[1].L['out'].value)

        rf_data['y_H2O'].append(model.reactive[1].y['H2O'].value)
        rf_data['y_CO'].append(model.reactive[1].y['CO'].value)
        rf_data['y_H2'].append(model.reactive[1].y['H2'].value)
        rf_data['y_CO2'].append(model.reactive[1].y['CO2'].value)
        rf_data['r_WGS'].append(model.reactive[1].kinetics_block.r_WGS.value)
        rf_data['r_FT'].append(model.reactive[1].kinetics_block.r_FT_total.value)

    rf_data_master[t] = rf_data


Working on T = 573.15 K
------------------------------------------------------------------------------------------------------------
Solved	|Reflux = 0.000 kmol/s	|Vapor = 0.190 kmol/s	|Distillate = 0.012 kmol/s	|Bottom = 0.001 kmol/s
Solved	|Reflux = 0.100 kmol/s	|Vapor = 0.192 kmol/s	|Distillate = 0.010 kmol/s	|Bottom = 0.002 kmol/s
Solved	|Reflux = 0.200 kmol/s	|Vapor = 0.194 kmol/s	|Distillate = 0.009 kmol/s	|Bottom = 0.003 kmol/s
Solved	|Reflux = 0.300 kmol/s	|Vapor = 0.195 kmol/s	|Distillate = 0.009 kmol/s	|Bottom = 0.003 kmol/s
Solved	|Reflux = 0.400 kmol/s	|Vapor = 0.196 kmol/s	|Distillate = 0.009 kmol/s	|Bottom = 0.003 kmol/s
Solved	|Reflux = 0.500 kmol/s	|Vapor = 0.197 kmol/s	|Distillate = 0.008 kmol/s	|Bottom = 0.003 kmol/s
Solved	|Reflux = 0.600 kmol/s	|Vapor = 0.198 kmol/s	|Distillate = 0.008 kmol/s	|Bottom = 0.003 kmol/s
Solved	|Reflux = 0.700 kmol/s	|Vapor = 0.198 kmol/s	|Distillate = 0.008 kmol/s	|Bottom = 0.003 kmol/s
Solved	|Reflux = 0.800 kmol/s	|Vapor = 0.199 kmol/

Solved	|Reflux = 1.300 kmol/s	|Vapor = 0.165 kmol/s	|Distillate = 0.022 kmol/s	|Bottom = 0.006 kmol/s
Solved	|Reflux = 1.400 kmol/s	|Vapor = 0.167 kmol/s	|Distillate = 0.021 kmol/s	|Bottom = 0.006 kmol/s
Solved	|Reflux = 1.500 kmol/s	|Vapor = 0.168 kmol/s	|Distillate = 0.021 kmol/s	|Bottom = 0.006 kmol/s
Solved	|Reflux = 1.600 kmol/s	|Vapor = 0.169 kmol/s	|Distillate = 0.021 kmol/s	|Bottom = 0.006 kmol/s
Solved	|Reflux = 1.700 kmol/s	|Vapor = 0.171 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.006 kmol/s
Solved	|Reflux = 1.800 kmol/s	|Vapor = 0.172 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.006 kmol/s
Solved	|Reflux = 1.900 kmol/s	|Vapor = 0.173 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.006 kmol/s
Solved	|Reflux = 2.000 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.020 kmol/s	|Bottom = 0.006 kmol/s

Working on T = 533.15 K
------------------------------------------------------------------------------------------------------------
Solved	|Reflux = 0.000 kmol/s	|Vapor = 0.124 kmol/

Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.025 kmol/s
Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.025 kmol/s
Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.025 kmol/s
Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.025 kmol/s
Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.025 kmol/s
Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.025 kmol/s
Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.025 kmol/s
Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.025 kmol/s
Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.025 kmol/s
Solved	|Reflux = 0.301 kmol/s	|Vapor = 0.174 kmol/s	|Distillate = 0.000 kmol/s	|Bo

Solved	|Reflux = 0.022 kmol/s	|Vapor = 0.268 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s
Solved	|Reflux = 0.022 kmol/s	|Vapor = 0.268 kmol/s	|Distillate = 0.000 kmol/s	|Bottom = 0.016 kmol/s


In [41]:
cnumber_range = list(range(1,30)) + list(range(31,57))

In [27]:
rf_data = rf_data_master[543.15]

In [46]:
def trans_cnumber(dic):
    molefraction = {}
    for i in range(1,57):
        molefraction[i] = []
    for i in m.COMP_ORG:
        molefraction[cal_cnumber(i)].append(np.array(dic[i]))
    for i in range(1,57):
        molefraction[i] = np.sum(molefraction[i],0)
    molefraction.pop(30)
    length = len(molefraction[1])
    tmp = {}
    for j in range(length):
        tmp[j] = []
        for i in cnumber_range:
            tmp[j].append(molefraction[i][j])
    return tmp

In [88]:
g_data = {}
d_data = {}
b_data = {}
r_data = {}

for t in Trange:
    g_data[t] = trans_cnumber(rf_data_master[t]['g'])
    d_data[t] = trans_cnumber(rf_data_master[t]['d'])
    b_data[t] = trans_cnumber(rf_data_master[t]['b'])
    r_data[t] = trans_cnumber(rf_data_master[t]['r'])

In [105]:
 def plot_distribution(temperature,index):
    fig, ax = plt.subplots(figsize=(16,9))
    ax.plot(cnumber_range,g_data[temperature][index],'go-')
    ax.plot(cnumber_range,d_data[temperature][index],'ro-')
    ax.plot(cnumber_range,b_data[temperature][index],'bo-')
    ax.plot(cnumber_range,r_data[temperature][index],'c-',alpha=0.7,linewidth=5)
    ax.set_yscale("log")
    ax.set_ylim(1e-12, 1)
    ax.legend(['Vapor','Distillate','Bottom'],fontsize=18)
    ax.set_title('T, Reflux {:.2f} kmol/s'.format(rf_data['Re'][index]),fontsize=18)

    ax.set_ylabel('Molar Flow (kmol/s)', color='K',fontsize=18)
    ax.set_xlabel('Carbon Number', color='K',fontsize=18)
    # ax.tick_params('y', colors='k',labelsize=18)
    # ax.tick_params('x', colors='k',labelsize=18)
    ax.grid()
    plt.show()

In [113]:
widgets.interact(plot_distribution, index = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Reflux:',), temperature=widgets.SelectionSlider(
    options=Trange,
    value=Trange[0],
    description='Temperature',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
));

interactive(children=(SelectionSlider(continuous_update=False, description='Temperature', options=(573.15, 563…